# **run load_data.ipynb BEFORE running this!**

In [10]:
import pickle
import os.path
import numpy as np
normalized_data_file_name = 'data_normalized'
with open(normalized_data_file_name + '.pickle', 'rb') as handle:
    spectrum, temperature = pickle.load(handle)
    print(f"read data from {normalized_data_file_name}.pickle")
print(f"shape of spectrum data: {spectrum.shape}")
print(f"shape of temperature data: {temperature.shape}")
print()
print(f"there are {temperature.shape[0]} spectrums")
print(f"each spectrum is {spectrum.shape[1]} long")

read data from data_normalized.pickle
shape of spectrum data: (6000, 10000)
shape of temperature data: (6000, 1)

there are 6000 spectrums
each spectrum is 10000 long


In [11]:
indices_file_name = 'cross_validation_resample=2_fold=5'
with open(indices_file_name + '.pickle', 'rb') as handle:
    train_indices, test_indices = pickle.load(handle)
    print(f"got indices from {indices_file_name}.pickle")  
print()
print(f"sets of training indices: {len(train_indices)}")
print(f"number of training indices per set: {len(train_indices[0])}")
print(f"sets of testing indices: {len(test_indices)}")
print(f"number of testing indices per set: {len(test_indices[0])}")

got indices from cross_validation_resample=2_fold=5.pickle

sets of training indices: 10
number of training indices per set: 4800
sets of testing indices: 10
number of testing indices per set: 1200


In [12]:
input_dimension = spectrum.shape[1]
print(f"input dimension is: {input_dimension}")

input dimension is: 10000


In [13]:
import torch
import torch.nn as nn
# change: input dim = 1000, output dim = 1 (temperature value)
class SiameseModel(torch.nn.Module):
    def __init__(self, device, input_dim=input_dimension):
        super().__init__()
        self.relu  = nn.ReLU()
        self.hidden_dim = 500
        self.linear1 = torch.nn.Linear(input_dim, self.hidden_dim)
        self.linear2 = torch.nn.Linear(self.hidden_dim, self.hidden_dim)
        self.linear3 = torch.nn.Linear(self.hidden_dim, 1)
        self.device = device
        self.to(device)
    def forward_siamese(self, x):
        y = self.linear3(self.relu(self.linear2(self.relu(self.linear1(x)))))
        return y
    def forward(self, input):
        # print(f"SiameseModel: input shape: {input.shape}")
        # in training, input shape is: 
        #   [batch size, number of inputs * input dim 1, input dim 2]
        #   e.g. [32, 2, 1000]
        # in actually using, input shape would be:
        #   [input dim 2] (in our example, input is 1*1000)
        # in classification, we don't need the network used to generated embedding
        #   we just need the final result, which is the network's 
        #   output through contrastive loss network
        # here, we DO need the output, which is just one number!
        if(input.dim() == 3): # <- training
            # input is tensor of [x1, x2]
            x1 = input[:,0,:] 
            x2 = input[:,1,:]
            # print(f"SiameseModel: ")
            # print(f"x1 = {x1}, shape{x1.shape}")
            # print(f"x2 = {x2}, shape{x2.shape}")
            y1 = self.forward_siamese(x1)
            y2 = self.forward_siamese(x2)
            # print(f"SiameseModel: y1 = {y1}, y2 = {y2}")
            # change: remove sigmoid, use y result as is
            # y = torch.sigmoid(y)
            difference = y1 - y2 # <- order matter, no absolute value!
            return difference
        else: # <- actually using: 
            # input is just x
            y = self.forward_siamese(input)
            return y

In [14]:
import random
# reference:
# https://colab.research.google.com/github/maticvl/dataHacker/
# blob/master/pyTorch/014_siameseNetwork.ipynb#scrollTo=gD1BFFm_z7aj
class SiameseDataset(torch.utils.data.TensorDataset):
    def __init__(self, X, y):
        # subset_X_IDs: e.g. only use mth to nth examples to train
        self.X, self.y = X, y
        self.indices = range(len(y))
    # length: number of elements in subset of X_IDs
    def __len__(self):
        return len(self.y)
    # get item by index in [m, n]
    def __getitem__(self, index): # assume index lies within subset_X_IDs
        # property: randomly sample two inputs,
        # return: the two inputs, 
        #   and binary label for whethery they have same class
        input_1_id = index
        input_1 = self.X[input_1_id]
        label_1 = self.y[input_1_id]
        input_2_id = random.choice(self.indices)
        input_2 = self.X[input_2_id]
        label_2 = self.y[input_2_id]
        difference = label_1 - label_2 # order matter!
        # turn into array BEFORE turning into tensor?
        #   going from list to tensor is slow
        #   going from array to tensor is fast
        # in this implementation, input (see initialization) already on cuda device
        # turning into array actually needs moving into cpu first
        # so just do: list of tensor -> tensor
        #   which is torch.stack
        return torch.stack([input_1, input_2]), difference

In [15]:
from torch.utils.data import DataLoader
from torch import optim
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class CalculateMSE():
    def __init__(self, net, n_epochs, batch_size):
        super().__init__()
        self.net = net
        #initialize some constants
        self.batch_size = 32
        self.learning_rate = 1e-4
        self.n_epochs = n_epochs
        self.net.apply(self.weights_init)   
    def weights_init(self,layer):
        if type(layer) == nn.Linear:
            nn.init.orthogonal_(layer.weight)
    def get_mse(self,train_data, train_label, test_data, test_label):
        train_set = SiameseDataset(
            torch.Tensor(train_data).to(device), 
            torch.Tensor(train_label).to(device))
        val_set = SiameseDataset(
            torch.Tensor(test_data).to(device), 
            torch.Tensor(test_label).to(device))
        loader_args = dict(batch_size=self.batch_size)
        train_loader = DataLoader(train_set, shuffle=True, drop_last=True, **loader_args)
        val_loader = DataLoader(val_set, shuffle=True, drop_last=True, **loader_args)
        tloss = []
        vloss = []
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.net.parameters(), lr=self.learning_rate) # weight_decay=0
        for epoch in range(0, self.n_epochs):
            # if epoch % 1000 == 0:
            #     print(f"epoch = {epoch}")
            epoch_train_loss=[]
            for i, data in enumerate(train_loader, 0):
                inputs, label = data
                y_pred = self.net(inputs.to(self.net.device))
                loss = criterion(y_pred, label.to(self.net.device))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                epoch_train_loss.append(loss.item())
            tloss.append(np.mean(epoch_train_loss))
            epoch_loss=[]
            for i, data in enumerate(val_loader, 0):
                with torch.no_grad():
                    inputs1, label1 = data
                    y_pred1 = self.net(inputs1.to(self.net.device))
                    loss1 = criterion(y_pred1, label1.to(self.net.device))
                    epoch_loss.append(loss1.item())
            vloss.append(np.mean(epoch_loss))
        return np.min(vloss), self.net

In [16]:
from pathlib import Path
from datetime import datetime
# change: turn into 10 right now for development, was 3000
n_epochs=1000
batch_size=32

PATH = 'model_siamese/'
Path(PATH).mkdir(parents=True, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# change: commented out: alraedy read in X, y data earlier
# response = pd.read_csv("1127_final_data/response.csv", header=None).values #input X
# spectra = pd.read_csv("1127_final_data/spectra.csv", header=None).values #ground truth label Y
# change: input dim = 1000
mdl = SiameseModel(device=device, input_dim=input_dimension)
losses = []
print(f"number of epochs: {n_epochs}, batch size: {batch_size}, device: {mdl.device}")

f = open("log_siamese.txt", "w")
f.write("train START: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
f.write(f"number of epochs: {n_epochs}, batch size: {batch_size}, device: {device}\n")
for i,(train,test) in enumerate(zip(train_indices,test_indices)):
    print(f"we are on fold no.{i}")
    train_data, train_label= spectrum[train],temperature[train]
    test_data, test_label= spectrum[test],temperature[test]
    mse_calculator = CalculateMSE(mdl,n_epochs,batch_size)
    loss,model = mse_calculator.get_mse(train_data, 
                                        train_label, 
                                        test_data, 
                                        test_label)
    losses.append(loss)
    print(f"\tloss: {loss}")
    print("\ttime: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        
    f.write(f"fold = {i}")
    f.write(f"\tloss: {loss}\n")
    f.write("\ttime: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") +"\n")

    torch.save(model.state_dict(), PATH+'model_'+str(i))
f.write("train END: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
f.close()

number of epochs: 1, batch size: 32, device: cuda
we are on fold no.0
	loss: 0.011823511367814767
	time: 2023-05-19 19:43:35
we are on fold no.1
	loss: 0.015360444607968265
	time: 2023-05-19 19:43:37
we are on fold no.2
	loss: 0.015916785476980982
	time: 2023-05-19 19:43:38
we are on fold no.3
	loss: 0.01806842487909504
	time: 2023-05-19 19:43:39
we are on fold no.4
	loss: 0.028112949098686914
	time: 2023-05-19 19:43:41
we are on fold no.5
	loss: 0.030776682687369553
	time: 2023-05-19 19:43:42
we are on fold no.6
	loss: 0.026183626337631327
	time: 2023-05-19 19:43:43
we are on fold no.7
	loss: 0.0132967991049628
	time: 2023-05-19 19:43:44
we are on fold no.8
	loss: 0.013775157830300363
	time: 2023-05-19 19:43:46
we are on fold no.9
	loss: 0.03180862787003453
	time: 2023-05-19 19:43:47


In [17]:
print(f"mean losses: {np.mean(losses)}, std: {np.std(losses)}")

mean losses: 0.020512300926084453, std: 0.007412684152474362


In [18]:
number_figures = 10
import matplotlib.pyplot as plt

indices = torch.randint(0,len(spectrum),(number_figures,)).unique()
for i in indices:
    print(f"we use {i}th example")
    # change: cast i to int, since pandas not work with torch.int64
    # changed: removed figure, since the output is just one number
    spec = np.asarray(spectrum[int(i)]).flatten()
    temp = np.asarray(temperature[int(i)]).flatten()
    # plt.figure(i)

    input = torch.Tensor(np.asarray(spectrum[int(i)])).to(model.device)
    print(f"input shape: {input.shape}, number of dimensions: {input.dim()}")
    prediction = model(input).detach().cpu().flatten()
    # plt.plot(prediction)
    print(f"\tthe prediction (normalized) is: {prediction.item()}")
    # plt.plot(temp)
    # plt.legend(['reconstruction','ground truth'])
    print(f"\tthe ground truth (normalized) is: {temp.item()}")


we use 344th example
input shape: torch.Size([10000]), number of dimensions: 1
	the prediction (normalized) is: -1.4608262777328491
	the ground truth (normalized) is: -1.5849058664490416
we use 621th example
input shape: torch.Size([10000]), number of dimensions: 1
	the prediction (normalized) is: -1.151224136352539
	the ground truth (normalized) is: -1.5849058664490416
we use 1125th example
input shape: torch.Size([10000]), number of dimensions: 1
	the prediction (normalized) is: -0.5595898628234863
	the ground truth (normalized) is: -0.8138705800684255
we use 1199th example
input shape: torch.Size([10000]), number of dimensions: 1
	the prediction (normalized) is: -0.5940682888031006
	the ground truth (normalized) is: -0.8138705800684255
we use 2835th example
input shape: torch.Size([10000]), number of dimensions: 1
	the prediction (normalized) is: 0.04583412781357765
	the ground truth (normalized) is: -0.2998470558146845
we use 3040th example
input shape: torch.Size([10000]), number 

# **skip cell 14 for now (it's in template file!)**